<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-6---The-Laplacian-Matrix" data-toc-modified-id="Chapter-6---The-Laplacian-Matrix-1">Chapter 6 - The Laplacian Matrix</a></span><ul class="toc-item"><li><span><a href="#6.1-The-Laplacian-Matrix-example" data-toc-modified-id="6.1-The-Laplacian-Matrix-example-1.1">6.1 The Laplacian Matrix example</a></span><ul class="toc-item"><li><span><a href="#6.2.3-Spectrum-of-L" data-toc-modified-id="6.2.3-Spectrum-of-L-1.1.1">6.2.3 Spectrum of L</a></span></li></ul></li><li><span><a href="#Example-6.10:-Basic-Graphs-and-their-algebraic-connectivity-(and-spectrum)" data-toc-modified-id="Example-6.10:-Basic-Graphs-and-their-algebraic-connectivity-(and-spectrum)-1.2">Example 6.10: Basic Graphs and their algebraic connectivity (and spectrum)</a></span></li><li><span><a href="#6.4-Appendix:-Community-Detection-via-algebraic-connectivity" data-toc-modified-id="6.4-Appendix:-Community-Detection-via-algebraic-connectivity-1.3">6.4 Appendix: Community Detection via algebraic connectivity</a></span></li></ul></li></ul></div>

In [ ]:
%matplotlib widget

# Import packages
import numpy as np
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
import networkx as nx

# For interactive graphs
import ipywidgets as widgets

# Import self defined functions
import lib  # General library

# Settings
custom_figsize= (6, 4) # Might need to change this value to fit the figures to your screen
custom_figsize_square = (5, 5) 

# Chapter 6 - The Laplacian Matrix

These Jupyter Notebook scripts contain some examples, visualization and supplements accompanying the book "Lectures on Network Systems" by Francesco Bullo http://motion.me.ucsb.edu/book-lns/. These scripts are published with the MIT license. **Make sure to run the first cell above to import all necessary packages and functions and adapt settings in case.** In this script it is necessary to execute cell by cell chronologically due to reocurring examples (Tip: Use the shortcut Shift+Enter to execute each cell). Most of the functions are kept in separate files to keep this script neat.

## 6.1 The Laplacian Matrix example


In this section we repeat the example from the book. The adjacancy Matrix $A$ is shown and how we retrieve the the Laplacian Matrix $L$ 

Note: This example occurred first in Chapter 3.5
Note: The adjacency Matrix can be determined via networkx as below and shown in Chapter 4.1. However, the Laplacian Matrix must be calculated via $L = D_{out} - A$, since there is no built in function for directed graphs in networkx yet.

In [ ]:
# Digraph Example from section 3.5 with weights, self_loops etc.
G_di = nx.DiGraph()
edges = [(1,2), (2,1), (2,4), (1,3), (3,5), (5,1), (5,5), (3,4), (5,4)]
weights = [3.7, 8.9, 1.2, 2.6, 2.3, 4.4, 4.4, 1.9, 2.7]
for edge, weight in zip(edges, weights):
    G_di.add_edge(*edge, weight=weight)
pos_di = {1:[0.1,0.2],2:[.4,.5],3:[.5,.2],4:[.8,.5], 5:[.9,.2]}  # Define position of nodes in digraph plot

# Plot first digraph again with weights visualization from section 3.5
fig, ax61 = plt.subplots(figsize=custom_figsize)
nx.draw_networkx(G_di, node_size=100, ax=ax61, pos=pos_di, connectionstyle='arc3, rad = 0.1')
labels = nx.get_edge_attributes(G_di,'weight')
nx.draw_networkx_edge_labels(G_di,pos=pos_di,edge_labels=labels, label_pos=0.2)

# Determining the adjacancy matrix via networkx
# This will always result in a sparse matrix, nodelist argument important to keep order of nodes
A_di = nx.linalg.graphmatrix.adjacency_matrix(G_di, nodelist=range(1, G_di.number_of_nodes()+1)).toarray()
print("Adjacency Matrix determined by Networkx:")
lib.matprint(A_di)

# Determining the Laplacian Matrix via the adjacency matrix from previous step
# This calculates the outdegree with the weights via networkx, convert it outdegrees to a diagonal array matrix
# Remember, the alternative way is to take the row sums
D_out_dict = dict(G_di.out_degree(range(1, G_di.number_of_nodes()+1), weight='weight'))  # Keeping the order of the nodes
D_out = np.diag(list(D_out_dict.values()))  # Note: Could also just take the rowsums of A, but working with networkx is shown here
L_di = D_out - A_di

print("\nLaplacian Matrix:")
lib.matprint(L_di)

### 6.2.3 Spectrum of L

As reviewied in the book, we can also look at the spectrum of $L$ and visualize the result with the Geršgorin Disks Theorem. Given a weighted digraph $G$ with Laplacian $L$, the eigenvalues of $L$ different from 0 have strictly-positive real part. 

In [ ]:
fig, ax623 = plt.subplots(figsize=custom_figsize_square)
lib.plot_gersgorin_disks(L_di, ax623, patch_spectrum=False)

## Example 6.10: Basic Graphs and their algebraic connectivity (and spectrum)

Similar to the basic graphs, their adjacency matrices and spectra from (Table 4.1), we present the algebraic connectivity and the Laplacian Matrix spectrum. Note that the algebraic connectivity value is presented in the legend of the second column subplots! 

In [ ]:
n_m = 6  # Can change this value
# Path Graph
G_path = nx.path_graph(n_m)
# Cycle Graph
G_cycle = nx.cycle_graph(n_m)
# Star Graph
G_star = nx.star_graph(n_m-1)
# Complete Graph
G_complete = nx.complete_graph(n_m)
# Complete bipartite Graph
G_bipartite = nx.complete_bipartite_graph(n_m//2, n_m//2)

all_basic_graphs = {
    "Path Graph": G_path,
    "Cycle Graph": G_cycle,
    "Star Graph":G_star,
    "Complete Graph": G_complete,
    "Complete bipartite Graph": G_bipartite,
    }
spectrums = [
    "$\{0\} \cup \{2(1-cos(\pi i / n))  | i ∈ \{1, . . . , n-1\}\}$",
    "$\{0\} \cup \{2(1-cos(2\pi i/n)) | i ∈ \{1, . . . , n-1\}\}$",
    "$\{ 0,1,...,1,n \}$",
    "$\{0,n,...,n\}$",
    "$\{0, m, ... , m, n, ..., n, m+n\}$"
]

alg_conn = [
    "$2(1-cos(\pi/n)) $",
    "$2(1-cos(2\pi/n)) $",
    "$1$",
    "$n$",
    "$min(n, m)$"
]

# Storing executable equations in a ord list:
alg_conn_func = {
      "Path Graph":  (lambda n, m: 2*(1-np.cos(np.pi/n))),
      "Cycle Graph":   (lambda n, m: 2*(1-np.cos(2*np.pi/n))),
      "Star Graph":  (lambda n, m: 1),
      "Complete Graph": (lambda n, m: n),
      "Complete bipartite Graph": (lambda n, m: min(m,n))
}

In [ ]:
# Plotting the graph itself and its spectrum, with additionally visualizing the algebraic connectivity 
fig, axs610 = plt.subplots(len(all_basic_graphs), 2, figsize=(custom_figsize[0]*1.0, custom_figsize[1]*4))
for count, (key, graph) in enumerate(all_basic_graphs.items()):
    # First subplot
    if key == "Complete bipartite Graph":
        nx.draw_networkx(graph, node_size=100, ax=axs610[count, 0], pos=nx.drawing.layout.bipartite_layout(graph, list(range(0, n_m//2))))
    else:
        nx.draw_networkx(graph, node_size=100, ax=axs610[count, 0], connectionstyle='arc3, rad = 0.1')
    axs610[count, 0].set_xlabel(key)
    # Plotting spectrum of L
    A_tmp = nx.linalg.graphmatrix.adjacency_matrix(graph, nodelist=range(0, graph.number_of_nodes())).toarray()
    D_out_dict_tmp = dict(graph.degree(range(0, graph.number_of_nodes())))  # Keeping the order of the nodes
    D_out_tmp = np.diag(list(D_out_dict_tmp.values()))
    L = D_out_tmp - A_tmp
    lib.plot_spectrum(L, axs610[count, 1]);
    axs610[count, 1].set(title=spectrums[count])
    # Algebraic connectivity
    alg_conn_value = alg_conn_func[key](n_m, n_m)
    axs610[count, 1].plot(alg_conn_value, 0, 'r.', ms=15, label = "Alg. connectivity:\n" + alg_conn[count] + " = " + "%.4g" % alg_conn_value)
    axs610[count, 1].legend()
    
    #axs610[count, 1].set_xlabel("Alg. conn: " + alg_conn[count] + " = " + "%.4g" % alg_conn_value)
fig.subplots_adjust(hspace=0.4)

## 6.4 Appendix: Community Detection via algebraic connectivity

In this section the Matlab Code example is implemented in Python below. Remember, that we are looking for the "bottleneck" of the graph by optimizing the size of the cut's minima. From the book:

To illustrate these concepts, we borrow an example problem with the corresponding Matlab code from (Gleich, 2006). We construct a randomly generated graph as follows. First, we partition $n$ = 1000 nodes in two groups $V_1$ and $V_2$ of sizes 450 and 550 nodes, respectively. Second, we connect any pair of nodes in the set $V_1$ (respectively $V2$) with probability 0.3 (respectively 0.2). Third and finally, any two nodes in distinct groups, $i ∈ V_1$ and $j ∈ V_2$, are connected with a probability of 0.1. The sparsity pattern of the associated adjacency matrix is shown in the left panel of Figure 6.6. No obvious partition is visible at first glance since the indices are not necessarily sorted, that is, $V_1$ is not necessarily {1, . . . , 450}. The second panel displays the sorted entries of the eigenvector $V_2$ showing a sharp transition between positive and negative entries. Finally, the third panel displays the correspondingly sorted adjacency matrix
Figure 6.6 is in the Table 6.2 below. 

In [ ]:
# for a given graph size, randomly assign the nodes to two groups
n = 1000
group_size = 450;
A = np.zeros([1000, 1000])
x = np.random.permutation(n) - 1  # Random permutation of indices for groups
group1 = x[0:group_size]
group2 = x[group_size:];

# assign probabilities of connecting nodes
# Note: a little bit adapted for better visualization
p_group1 = 0.4
p_group2 = 0.3
p_between_groups = 0.2

#construct adjacency matrix
A[np.ix_(group1,group1)] = (np.random.rand(group_size, group_size) < p_group1) * 1  # Ensure cast to integer
A[np.ix_(group2,group2)] = (np.random.rand(n-group_size,n-group_size) < p_group2) * 1
A[np.ix_(group1,group2)] = (np.random.rand(group_size, n-group_size) < p_between_groups) * 1
# Ensure symmetry by copying the just created upper triangle part
A = np.triu(A,1)
A = A + A.T

# construct Laplacian and its spectrum
L = np.diag(np.sum(A, 1)) - A;

# Retrieve both 2nd smallest eigenvalues
D, V = sla.eigs(L, 2, which='SM')

# Get indices for sorting the eigenvector of the corresponding fiedler eigenvalue
V_sort_ind = np.argsort(V[:, 1])

# Init plot
fig, axs64 = plt.subplots(3, 1, figsize=(custom_figsize[0]*1.2, custom_figsize[1]*3))

# Plot binary matrix
lib.plot_matrix_binary(A, axs64[0])
axs64[0].set_xlabel("$A$")

# Plot the eigenvector values sorted by magnitude
axs64[1].plot(np.sort(V[:, 1]))
axs64[1].set_aspect(1 / axs64[1].get_data_ratio())  # Workaround to make it square without equal axis ticks
axs64[1].set_xlabel(r"$\tilde{v}_2$")

# Plot the adjacency matrix sorted by the eigenvector
lib.plot_matrix_binary(A[np.ix_(V_sort_ind,V_sort_ind)], axs64[2])
axs64[2].set_xlabel(r"$\widetilde{A}$");